In [1]:
from __future__ import print_function

#This script implements pedestrian traffic forecast using Gaussian process regression with squared exponential kernel

#The inputs required are: date, hour, intersection ID, holiday (Boolean) frecast horizon

#Requirements: 'gprPedsMdl.sav', 'gprCycsMdl' and 'Locations.csv'

In [2]:
%load_ext autoreload
%autoreload

import ipywidgets as widgets
from ipywidgets import GridspecLayout, Button, Layout
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMapWithTime
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
file = open("pedestrianTraffic.jpg", "rb")
image = file.read()
pedsImage=widgets.Image(
    value=image,
    format='jpg',
    layout=Layout(width='100%', height='400px')
)
display(pedsImage)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xfe\x00;CREATOR: gd-jpeg v1.0…

In [4]:
info = widgets.Button(
    description='This tool predicts pedestrian and cyclist traffic at selected intersections at Downtown Toronto',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width='100%', height='40px')
)
display(info)

Button(button_style='info', description='This tool predicts pedestrian and cyclist traffic at selected interse…

In [5]:
#Reading saved files

filename = 'Locations.sav'
Locations = pickle.load(open(filename, 'rb'))

filename = 'gprPedsMdl.sav'
gprPedsMdl = pickle.load(open(filename, 'rb'))

filename = 'gprCycsMdl.sav'
gprCycsMdl = pickle.load(open(filename, 'rb'))

In [ ]:
#Hoidays in Toronto in 2019

holidayDates = ['2019-01-01', '2019-02-14', '2019-02-18', '2019-03-17',
          '2019-04-19', '2019-04-22', '2019-05-12', '2019-05-20',
          '2019-06-16', '2019-07-01', '2019-08-05', '2019-09-02',
          '2019-10-14', '2019-10-31', '2019-12-25', '2019-12-26']

HolidayCal = pd.DataFrame(holidayDates, columns = ['dates']);
HolidayCal = pd.to_datetime(HolidayCal, dayfirst = True)

In [6]:
#Converting widget outputs to appropriate data types for get_forecast() and get_maps()

def get_param(ForecastDate,StartHour,forecastHorizon,InterSection,RoadUser):
    
    FirstDate= ForecastDate.value

    FirstTime=int(StartHour.value);

    Horizon=forecastHorizon.value

    testStart = datetime(FirstDate.year, FirstDate.month, FirstDate.day)
    testStart = testStart + pd.Timedelta(hours=FirstTime);
    
    testEnd = testStart + pd.Timedelta(hours=Horizon-1);
    
    if InterSection.value == 'Adelaide-Bay':
        IntID= 0;
    elif InterSection.value == 'Adelaide-Jarvis':
        IntID= 1;
    elif InterSection.value == 'Front-Bathurst':
        IntID= 2;
    elif InterSection.value == 'King-Portland':
        IntID= 3;
    elif InterSection.value == 'King-Peter':
        IntID= 4;
    elif InterSection.value == 'King-Bay':
        IntID= 5;
    elif InterSection.value == 'King-Yonge':
        IntID= 6;
    elif InterSection.value == 'King-Church':
        IntID= 7;
    elif InterSection.value == 'King-Jarvis':
        IntID= 8;
    elif InterSection.value == 'Queen-Bathurst':
        IntID= 14;
        print('We don\'t have historical data for this intersection, but we will give you an estimate...')
    elif InterSection.value == 'Richmond-Spadina':
        IntID= 9;
    elif InterSection.value == 'Richmond-Bay':
        IntID= 10;
    elif InterSection.value == 'Wellington-Blue_Jays':
        IntID= 11;
    elif InterSection.value == 'Wellington-Bay':
        IntID= 12;
    elif InterSection.value == 'King-John':
        IntID= 13;
        
    if RoadUser.value == 'Pedestrians':
        RoadUserID = 1;
    elif RoadUser.value == 'Cyclists':
        RoadUserID = 2;
        
    return RoadUserID, InterSection.value, IntID, testStart, testEnd, Horizon

In [7]:
def get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon):
    
    if RoadUserID == 1:
        gprMdl = gprPedsMdl
    elif RoadUserID == 2: 
        gprMdl = gprCycsMdl
    
    dateTimeIdx = pd.date_range(start = testStart, periods = Horizon, freq = 'H');
      
    hour = dateTimeIdx.hour; dayOfWeek = dateTimeIdx.dayofweek;
    
    dates = pd.DataFrame(dateTimeIdx.date, columns=['dates']);
    dates = dates['dates']
    Holidays = dates.isin(HolidayCal['dates'].dt.date)
    Holidays[Holidays == True] = 1; Holidays[Holidays == False]=0;
    
    lat = Locations.iloc[IntID, 0]; lng = Locations.iloc[IntID, 1]
    
    xTest = pd.DataFrame();
    xTest['Hour'] = hour; xTest['DayOfWeek'] = dayOfWeek; xTest['isHoliday'] = Holidays;
    xTest['lat'] = lat*np.ones([len(xTest),1]); 
    xTest['lng'] = lng*np.ones([len(xTest),1]);
       
    xTest.index = dateTimeIdx;
    
    yPred, yStd = gprMdl.predict(xTest, return_std=True)
    yPred = yPred.astype('int'); yPred[yPred<0]=0
    yPred = pd.DataFrame(yPred, columns =['forecast']); yPred.index = dateTimeIdx
    yPred = yPred['forecast']
    
    yLow = yPred - 3*yStd; yLow = yLow.astype('int'); yLow[yLow<0] = 0
    yHigh = yPred + 3*yStd; yHigh = yHigh.astype('int');
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yPred, name = 'Forecast'))
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yLow, 
                            mode = 'lines',
                            line = dict(width = 0.1),
                            name = ''))
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yHigh, fill = 'tonexty', 
                            mode = 'lines',
                            line = dict(width=0.1),
                            name = '99.7% confidence interval'))
    fig.update_xaxes(title_text = 'Time')
    if RoadUserID == 1: 
        fig.update_yaxes(title_text='Pedestrian volume')
        fig.update_layout(title_text = 'Pedestrian traffic forecast at '+ Intersection + ' Intersection')
    elif RoadUserID == 2:
        fig.update_yaxes(title_text='Cyclist volume')
        fig.update_layout(title_text = 'Cyclist traffic forecast at '+ Intersection + ' Intersection')
    fig.show()

In [8]:
def get_maps(RoadUserID, IntID, testStart, testEnd, Horizon):
    
    if RoadUserID == 1:
        gprMdl = gprPedsMdl
    elif RoadUserID == 2: 
        gprMdl = gprCycsMdl
    
    IntNames = ['Adelaide-Bay', 'Adelaide-Jarvis', 'Front-Bathurst', 'King-Portland',
               'King-Peter', 'King-Bay', 'King-Yonge', 'King-Church', 'King-Jarvis',
               'Richmond-Spadina','Richmond-Bay','Wellington-Blue_Jays',
                'Wellington-Bay','King-John','Queen-Bathurst'];
    
    IntLoc = Locations.iloc[13,:] #13 is selected because it is the middlemost-located intersection 
    #to allow easy visualisation of all other intersections
    
    trafficMap = folium.Map(location= IntLoc, zoom_start = 15)
    
    dateTimeIdx = pd.date_range(start = testStart, periods = Horizon, freq = 'H');
      
    hour = dateTimeIdx.hour; dayOfWeek = dateTimeIdx.dayofweek;
    
    dates = pd.DataFrame(dateTimeIdx.date, columns=['dates']);
    dates = dates['dates']
    Holidays = dates.isin(HolidayCal['dates'].dt.date)
    Holidays[Holidays == True] = 1; Holidays[Holidays == False]=0;
    
    lat = Locations.iloc[IntID, 0]; lng = Locations.iloc[IntID, 1]
    
    xTest = pd.DataFrame();
    xTest['Hour'] = hour; xTest['DayOfWeek'] = dayOfWeek; xTest['isHoliday'] = Holidays;
       
    xTest.index = dateTimeIdx;
    
    IntForecasts = [ [] for i in range(Horizon) ]
    for IntID in range(len(Locations)):

        lat = Locations.iloc[IntID, 0]; lng = Locations.iloc[IntID, 1]

        xTest['lat']= lat*np.ones([len(xTest),1]);
        xTest['lng']= lng*np.ones([len(xTest),1]);

        yPred, yStd = gprMdl.predict(xTest, return_std=True)
        yPred = yPred.astype('int'); yPred[yPred<0]=0
        yPred = pd.DataFrame(yPred, columns =['forecast']); yPred.index = dateTimeIdx;
        yPred = yPred['forecast']
        
        IntLoc = Locations.iloc[IntID,:]
        
        folium.Marker(location = IntLoc, popup = IntNames[IntID]).add_to(trafficMap) 
        
        for i in range(Horizon):
            IntForecasts[i].append([IntLoc.values[0], IntLoc.values[1], yPred.values[i]])
    
    HeatMapWithTime(IntForecasts, radius=12, auto_play = True,
                    gradient={0: 'blue', 0.2: 'lime', 0.4: 'orange', 0.6: 'red', 1: 'black'}, 
                    min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(trafficMap)
    display(trafficMap)

In [9]:
grid = GridspecLayout(1, 4, grid_gap="10px")

In [10]:
InterSection = widgets.Dropdown(
    options = ['Adelaide-Bay', 'Adelaide-Jarvis','Front-Bathurst', 
             'King-Bay', 'King-Church', 'King-Jarvis',
             'King-John', 'King-Peter', 'King-Portland', 
             'King-Yonge', 'Queen-Bathurst', 'Richmond-Bay',
             'Richmond-Spadina', 'Wellington-Bay', 'Wellington-Blue_Jays'],
    description = 'Intersection:',
    value = 'Adelaide-Bay',
    disabled = False,
)
grid[0,1] = InterSection

In [11]:
RoadUser = widgets.Dropdown(
    options = ['Pedestrians', 'Cyclists'],
    description = 'Road user:',
    value = 'Pedestrians',
    disabled = False,
)
grid[0,2] = RoadUser

In [12]:
display(grid)

GridspecLayout(children=(Dropdown(description='Intersection:', layout=Layout(grid_area='widget001'), options=(…

In [13]:
grid = GridspecLayout(1, 3, grid_gap = "10px")

In [14]:
style = {'description_width': 'initial'}
ForecastDate = widgets.DatePicker(
    style = style,
    description = 'Forecast start date:',
    value = pd.to_datetime('2019-08-08'),
    disabled = False
)
grid[0,0] = ForecastDate

In [15]:
style = {'description_width': 'initial'}
StartHour = widgets.Dropdown(
    options = ['0','1', '2', '3','4','5','6','7','8','9','11','12',
            '13','14','15','16','17','18','19','20','21','22','23','24'],
    style = style,
    description = 'Forecast start hour:',
    value = '0',
    disabled = False,
)
grid[0,1] = StartHour

In [16]:
style = {'description_width': 'initial'}
forecastHorizon = widgets.IntText(
    value = 24,
    style = style,
    description = 'Forecast horizon in hours:',
    disabled = False
)
grid[0,2] = forecastHorizon

In [17]:
display(grid)

GridspecLayout(children=(DatePicker(value=Timestamp('2019-08-08 00:00:00'), description='Forecast start date:'…

In [18]:
grid = GridspecLayout(1, 4, grid_gap = "10px")

In [19]:
style = {'description_width': 'initial'}
ts_button = widgets.Button(description = "Click here for graph",
                          button_style = 'success')
ts_output = widgets.Output()
grid[0,1] = ts_button

def on_ts_button_clicked(b):
    with ts_output:
        print('This might take a while...')
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        
        get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon)
        print('Completed!')
        ts_output.clear_output(wait = True)
        
ts_button.on_click(on_ts_button_clicked)

In [20]:
style = {'description_width': 'initial'}
maps_button = widgets.Button(description = "Click here for map",
                            button_style = 'success',
                            style = style)
maps_output = widgets.Output()
grid[0,2] = maps_button

def on_maps_button_clicked(b):
    with maps_output:
        print('This might take a while...')
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        get_maps(RoadUserID, testStart, testStart, testEnd, Horizon)
        print('Completed!')
        maps_output.clear_output(wait = True)
        
maps_button.on_click(on_maps_button_clicked)

In [21]:
info = widgets.Button(
    description = 'Click below for time-series forecasts or heatmap evolving with time',
    disabled = True,
    button_style = ''
    layout = Layout(width = '100%', height = '40px')
)
display(info)

Button(description='Click below for time-series forecasts or heatmap evolving with time', disabled=True, layou…

In [22]:
display(grid)
display(ts_output)
display(maps_output)

GridspecLayout(children=(Button(button_style='success', description='Click here for graph', layout=Layout(grid…

Output()

Output()